In [12]:
import pathlib
import json 
import albumentations as A

In [13]:
artf_tshirts_train_json_path = "/home/tlips/Documents/aRTF-Clothes-dataset/data/aRTFClothes/tshirts-train.json"
artf_tshirts_test_json_path = "/home/tlips/Documents/aRTF-Clothes-dataset/data/aRTFClothes/tshirts-test.json"
artf_tshirts_val_json_path = "/home/tlips/Documents/aRTF-Clothes-dataset/data/aRTFClothes/tshirts-val.json"

In [14]:

# read each annotation. 
# get the min and max x coord of any keypoint

# pad this to a square image

# save the image and the new annotations.


from albumentations.core.transforms_interface import BoxInternalType
from numpy import ndarray


class KeypointsCenteredHorizontalCrop(A.DualTransform):
    def __init__(self, always_apply=False, p=1.0):
        super(KeypointsCenteredHorizontalCrop, self).__init__(always_apply, p)


    def apply(self, img,kp, **params):
        x_min, x_max = self._get_min_and_max_x_coords(kp, params['image_height'], params['image_width'])
        if len(img.shape) == 3:
            return img[:, x_min:x_max, :]
        else:
            # mask 
            return img[:,x_min:x_max]
    

    def _get_min_and_max_x_coords(self, keypoints, image_height, image_width, margin = 10):
        x_coords = [kp[0] for kp in keypoints]
        x_min, x_max = min(x_coords), max(x_coords)
        x_min,x_max = round(x_min), round(x_max)

        # if possible, add a small margin around the keypoints to avoid 'border keypoints'
        xmin = max(0, x_min - margin)
        xmax = min(image_width, x_max + margin)

        padding = (image_height - (x_max - x_min)) // 2
        if padding < 0:
            padding = 0
        
        x_min = max(0, x_min - padding)
        x_max = min(image_width, x_max + padding)
        return x_min, x_max

    def apply_to_keypoints(self, keypoints, **params):
        x_min, x_max = self._get_min_and_max_x_coords(keypoints, params['image_height'], params['image_width'])
        new_keypoints  = []
        for kp in keypoints:
            new_keypoints.append([kp[0] - x_min, kp[1], kp[2], kp[3]])

        return new_keypoints
    
    def apply_to_bbox(self, bbox, kp,**params):
        x_min, x_max = self._get_min_and_max_x_coords(kp, params['image_height'], params['image_width'])
        
        bbox_x_min = bbox[0]
        bbox_x_max = bbox[2]
        bbox_y_min = bbox[1]
        bbox_y_max = bbox[3]

        new_bbox_x_min = (bbox_x_min * params['image_width'] - x_min) / (x_max - x_min)
        new_bbox_x_max = (bbox_x_max * params['image_width'] - x_min) / (x_max - x_min)
        
        return (new_bbox_x_min, bbox_y_min, new_bbox_x_max, bbox_y_max)
    
    def get_params_dependent_on_targets(self, params):
        # hack to use 'target' as param for other targets
        kp = params['keypoints']
        image_height, image_width = params['image'].shape[:2]
        return {'kp': kp, 'image_height': image_height, 'image_width': image_width}
    
    @property
    def targets_as_params(self):
        return ['image', 'keypoints', 'bboxes']

In [15]:
transforms = [
    KeypointsCenteredHorizontalCrop(),
    A.Resize(512,512)
]

print(KeypointsCenteredHorizontalCrop().targets_as_params)


['image', 'keypoints', 'bboxes']


In [16]:
from airo_dataset_tools.coco_tools.transform_dataset import apply_transform_to_coco_dataset
from airo_dataset_tools.data_parsers.coco import CocoKeypointsDataset

In [17]:
artf_tshirts_train_dataset = CocoKeypointsDataset(**json.load(open(artf_tshirts_train_json_path)))
artf_tshirts_test_dataset = CocoKeypointsDataset(**json.load(open(artf_tshirts_test_json_path)))
artf_tshirts_val_dataset = CocoKeypointsDataset(**json.load(open(artf_tshirts_val_json_path)))


print(len(artf_tshirts_val_dataset.annotations))

42


In [18]:
transformed_annotations = apply_transform_to_coco_dataset(
    transforms,
    coco_dataset=artf_tshirts_train_dataset,
    image_path = str(pathlib.Path(artf_tshirts_train_json_path).parent),
    target_image_path= str(pathlib.Path(artf_tshirts_train_json_path).parents[1] / "tshirts-train-kpcentercropped"),
)
with open(str(pathlib.Path(artf_tshirts_train_json_path).parents[1] / "tshirts-train-kpcentercropped"/ "annotations.json"), "w") as f:
    json.dump(transformed_annotations.model_dump(exclude_none=True), f)



Transforming keypoints = True
Transforming bbox = True
Transforming segmentation = True


100%|██████████| 168/168 [00:15<00:00, 11.19it/s]


In [19]:
transformed_annotations = apply_transform_to_coco_dataset(
    transforms,
    coco_dataset=artf_tshirts_test_dataset,
    image_path = str(pathlib.Path(artf_tshirts_test_json_path).parent),
    target_image_path= str(pathlib.Path(artf_tshirts_test_json_path).parents[1] / "tshirts-test-kpcentercropped"),
)
with open(str(pathlib.Path(artf_tshirts_test_json_path).parents[1] / "tshirts-test-kpcentercropped"/ "annotations.json"), "w") as f:
    json.dump(transformed_annotations.model_dump(exclude_none=True), f)

Transforming keypoints = True
Transforming bbox = True
Transforming segmentation = True


100%|██████████| 400/400 [00:36<00:00, 10.94it/s]


In [20]:
transformed_annotations = apply_transform_to_coco_dataset(
    transforms,
    coco_dataset=artf_tshirts_val_dataset,
    image_path = str(pathlib.Path(artf_tshirts_val_json_path).parent),
    target_image_path= str(pathlib.Path(artf_tshirts_val_json_path).parents[1] / "tshirts-val-kpcentercropped"),
)
with open(str(pathlib.Path(artf_tshirts_val_json_path).parents[1] / "tshirts-val-kpcentercropped"/ "annotations.json"), "w") as f:
    json.dump(transformed_annotations.model_dump(exclude_none=True), f)

Transforming keypoints = True
Transforming bbox = True
Transforming segmentation = True


100%|██████████| 42/42 [00:03<00:00, 10.96it/s]
